In [159]:
#importing langchain modules

from langchain.document_loaders import UnstructuredExcelLoader
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain import OpenAI, VectorDBQA
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.document_loaders import TextLoader
import tiktoken
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.callbacks import get_openai_callback
#importing the os

from config import OPENAI_API_KEY

In [212]:
#loading the excel files
loader = UnstructuredExcelLoader(
    "excel/faculty.xlsx"
)

docs = loader.load()

loader = PyPDFLoader(
    "prof.pdf"
)

docs = loader.load()



In [213]:
text_splitter_character = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100
)

 
#docs = TextLoader('file.txt').load()
#embedding
all_splits = text_splitter_character.split_documents(docs)

In [214]:
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    token  = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(token)
token_counts = [tiktoken_len(doc.page_content) for doc in all_splits]
print(sum(token_counts))

4856


In [215]:
persist_directory = "db"

# create the open-source embedding function
vectordb = Chroma.from_documents(documents=all_splits,embedding=OpenAIEmbeddings(),
        persist_directory=persist_directory)

#

In [216]:
retrieval = vectordb.as_retriever(search_kwargs={"k":2})
#docs = retrieval.get_relevant_documents("Who is Rucha Joshi")


In [217]:

qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retrieval,
        return_source_documents=True, verbose=True)
print(qa_chain("Who is Dr. Manoj Kannan"))





> Entering new  chain...

> Finished chain.
{'query': 'Who is Dr. Manoj Kannan', 'result': ' Dr. Manoj Kannan is the Associate Dean, Student Life, and Associate Professor at Plaksha University. His area of expertise is discipline-based education research and his past associations include Associate Professor at Indian Institute of Technology Kharagpur and Post-doctoral Fellow at University of Texas Medical School, Houston.', 'source_documents': [Document(page_content='>CTO of Smantenna Ltd \n \nName: Dr. K. Gopinath \nDesignation: Professor, Plaksha University \nArea of Expertise: \nSystems research: OS, security \nResearch Interests: \n>Operating Systems \n>Cybersecurity and Privacy \n>Applied AI (for computer systems and for societal good) \nEducation: \n>Ph.D., Stanford University, USA \nPast Associations: \n>Professor, Computer Science & Automation, Indian Institute of Science, Bangalore (till July 31, 2021) \n \nName: Dr. Kriti Khanna \nDesignation: Assistant Professor, Plaksha U

In [218]:
#To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

#delete the directory

In [219]:
!rm -rf db/

In [220]:
!unzip db.zip

Archive:  db.zip
   creating: db/
   creating: db/index/
  inflating: db/index/id_to_uuid_8be6aa5e-3b16-4c1d-af03-e5940a1589fb.pkl  
  inflating: db/index/uuid_to_id_8be6aa5e-3b16-4c1d-af03-e5940a1589fb.pkl  
  inflating: db/index/index_metadata_8be6aa5e-3b16-4c1d-af03-e5940a1589fb.pkl  
  inflating: db/index/index_8be6aa5e-3b16-4c1d-af03-e5940a1589fb.bin  
  inflating: db/chroma-embeddings.parquet  
  inflating: db/chroma-collections.parquet  


In [221]:
#importing langchain modules

from langchain.document_loaders import UnstructuredExcelLoader
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain import OpenAI, VectorDBQA
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.document_loaders import TextLoader
import tiktoken
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.callbacks import get_openai_callback
#importing the os

from config import OPENAI_API_KEY

In [223]:
persist_directory = "db"

# create the open-source embedding function
vectordb2 = Chroma(embedding_function=OpenAIEmbeddings(),
        persist_directory=persist_directory,collection_name="state-of-union")

In [224]:
retrieval = vectordb.as_retriever()

In [225]:
state_of_union = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retrieval)

In [226]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re
from getpass import getpass

In [227]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

In [228]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [229]:
output_parser = CustomOutputParser()

In [230]:
template = """
Based the New question use {tools} to check plaksha professors and founders who have experience in the pinput by the student

You Thought:
Action:
Action Input:
Observation
Use this [{tool_names}] to check for the professor

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}
"""



In [231]:
llm=OpenAI(temperature=0)

In [232]:
from pydantic import BaseModel, Field

class DocumentInput(BaseModel):
    question: str = Field()


llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [233]:
tools = [
        Tool(
            args_schema=DocumentInput,
            name="plaksha",
            description=f"useful all the plaksha",
            func=state_of_union,
        )
]

In [234]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [235]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [236]:
tool_names = [tool.name for tool in tools]
llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo-0613",
)
# agent = LLMSingleActionAgent(
#     llm_chain=llm_chain, 
#     output_parser=output_parser,
#     stop=["\nObservation:"], 
#     allowed_tools=tool_names
# )


agent = initialize_agent(
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True,
)

In [237]:
agent({"input":"Hi"})



> Entering new  chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'Hi', 'output': 'Hello! How can I assist you today?'}

In [238]:
agent({"input":"Which professor has experience in DNA"})



> Entering new  chain...

Invoking: `plaksha` with `{'question': 'Which professor has experience in DNA?'}`


{'query': 'Which professor has experience in DNA?', 'result': "\nI don't know."}I'm sorry, but I don't have information about which professor has experience in DNA.

> Finished chain.


{'input': 'Which professor has experience in DNA',
 'output': "I'm sorry, but I don't have information about which professor has experience in DNA."}

In [239]:
agent({"input":"Which professor has experience in data science"})



> Entering new  chain...

Invoking: `plaksha` with `{'question': 'Which professor has experience in data science?'}`


{'query': 'Which professor has experience in data science?', 'result': "\nI don't know."}I'm sorry, but I don't have information about which professor has experience in data science.

> Finished chain.


{'input': 'Which professor has experience in data science',
 'output': "I'm sorry, but I don't have information about which professor has experience in data science."}

In [187]:
agent({"input":"Which professor should I meet"})



> Entering new  chain...
It depends on what you need to discuss or ask. Could you please provide more information about the topic or subject you need assistance with?

> Finished chain.


{'input': 'Which professor should I meet',
 'output': 'It depends on what you need to discuss or ask. Could you please provide more information about the topic or subject you need assistance with?'}

In [188]:
agent({"input":"Data science"})



> Entering new  chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


Data science is a multidisciplinary field that uses scientific methods, processes, algorithms, and systems to extract knowledge and insights from structured and unstructured data. It combines techniques from mathematics, statistics, computer science, and domain knowledge to analyze and interpret complex data sets.

Data scientists use various tools and programming languages such as Python, R, and SQL to collect, clean, and analyze data. They apply statistical models, machine learning algorithms, and data visualization techniques to uncover patterns, make predictions, and derive actionable insights from the data.

Data science has applications in various industries such as finance, healthcare, marketing, and technology. It is used to solve complex problems, make data-driven decisions, and drive innovation. Some common tasks performed by data scientists include data cleaning and preprocessing, exploratory data analysis, predictive modeling, and data visualization.

Overall, data science 

{'input': 'Data science',
 'output': "Data science is a multidisciplinary field that uses scientific methods, processes, algorithms, and systems to extract knowledge and insights from structured and unstructured data. It combines techniques from mathematics, statistics, computer science, and domain knowledge to analyze and interpret complex data sets.\n\nData scientists use various tools and programming languages such as Python, R, and SQL to collect, clean, and analyze data. They apply statistical models, machine learning algorithms, and data visualization techniques to uncover patterns, make predictions, and derive actionable insights from the data.\n\nData science has applications in various industries such as finance, healthcare, marketing, and technology. It is used to solve complex problems, make data-driven decisions, and drive innovation. Some common tasks performed by data scientists include data cleaning and preprocessing, exploratory data analysis, predictive modeling, and d

In [189]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

KeyError: 'agent'

In [ ]:
agent_executor.run("hi")

NameError: name 'agent_executor' is not defined

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory=ConversationBufferWindowMemory(k=2)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

KeyError: 'agent'

In [ ]:
agent_executor.run("Hello")



> Entering new  chain...


OutputParserException: Could not parse LLM output: `
Answer: Hi there! How can I help you?`

In [240]:
prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

ValidationError: 1 validation error for PromptTemplate
__root__
  Invalid prompt schema; check for mismatched or missing input parameters. 'tools' (type=value_error)

In [ ]:
llm_chain = LLMChain(
    llm=OpenAI(temperature=0),
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [ ]:
tools = [
   
    Tool(
        name="Plaksha",
        func=state_of_union.run,
        description="use all time",
    )
]

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [ ]:
agent = initialize_agent(tools, llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"), agent=AgentType.OPENAI_FUNCTIONS, verbose=True,    agent_kwargs=agent_kwargs,
    memory=memory,)

In [ ]:
agent.run("Hi")



> Entering new  chain...
Hello! How can I assist you today?

> Finished chain.


'Hello! How can I assist you today?'

In [ ]:
agent.run("Plaksha Professor with c++ knowledge")



> Entering new  chain...

Invoking: `Plaksha` with `c++`



C++ is a high-level programming language that is used to develop applications and software programs. It is one of the most popular programming languages and is used in many different industries.Plaksha offers a wide range of professors with expertise in C++. They can provide guidance and support in learning and mastering the language. Whether you are a beginner or an advanced learner, Plaksha professors can help you enhance your C++ skills and knowledge.

> Finished chain.


'Plaksha offers a wide range of professors with expertise in C++. They can provide guidance and support in learning and mastering the language. Whether you are a beginner or an advanced learner, Plaksha professors can help you enhance your C++ skills and knowledge.'

In [ ]:
agent.run("I want to learn about build applications for mac user and I will want a roadmap")



> Entering new  chain...

Invoking: `mainPrompt` with `build applications for mac user`




> Entering new  chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and execute these projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well. It suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validati

'Thank you for providing the information. Based on your interest in building applications for Mac users, here is a roadmap that you can follow:\n\n1. Learn Swift Programming Language:\n   - Take online courses or tutorials to learn the basics of Swift, the programming language used for developing macOS applications.\n   - Practice writing Swift code and familiarize yourself with the syntax and concepts.\n\n2. Understand macOS Development:\n   - Study the macOS development ecosystem and frameworks, such as Cocoa and Cocoa Touch.\n   - Learn about the macOS App Lifecycle and UI design principles specific to Mac applications.\n\n3. Explore Xcode:\n   - Get familiar with Xcode, the integrated development environment (IDE) for macOS app development.\n   - Learn how to create projects, manage dependencies, and use debugging tools.\n\n4. User Interface Design:\n   - Study macOS Human Interface Guidelines (HIG) to understand the principles of designing user-friendly and visually appealing Mac 

In [ ]:
agent.run("which professor can I meet to learn about DNA")



> Entering new  chain...

Invoking: `Plaksha` with `DNA`


 DNA stands for deoxyribonucleic acid. It is a molecule found in all living organisms and contains genetic information necessary for development and growth.To learn more about DNA, you can meet with Professor Rakesh Mishra. He is a renowned expert in the field of genetics and genomics. Professor Mishra has extensive knowledge and experience in studying DNA and its role in various biological processes. He would be a great resource to learn more about DNA and its significance in the field of biology.

> Finished chain.


'To learn more about DNA, you can meet with Professor Rakesh Mishra. He is a renowned expert in the field of genetics and genomics. Professor Mishra has extensive knowledge and experience in studying DNA and its role in various biological processes. He would be a great resource to learn more about DNA and its significance in the field of biology.'

In [ ]:
agent.run("on creating app for mac user")



> Entering new  chain...
Creating an app for Mac users involves developing a software application that is compatible with the macOS operating system. Here are the steps you can follow to create an app for Mac users:

1. Choose a Development Platform: Decide on the development platform you want to use for creating the app. You can choose from options like Swift, Objective-C, or cross-platform frameworks like React Native or Flutter.

2. Set Up Development Environment: Install Xcode, which is the integrated development environment (IDE) for macOS. Xcode provides all the necessary tools and resources for developing Mac apps.

3. Design the User Interface: Use Interface Builder in Xcode to design the user interface of your app. You can drag and drop UI elements, customize their properties, and create a visually appealing interface.

4. Write Code: Use the programming language of your choice (Swift or Objective-C) to write the code for your app's functionality. This includes implementing 

"Creating an app for Mac users involves developing a software application that is compatible with the macOS operating system. Here are the steps you can follow to create an app for Mac users:\n\n1. Choose a Development Platform: Decide on the development platform you want to use for creating the app. You can choose from options like Swift, Objective-C, or cross-platform frameworks like React Native or Flutter.\n\n2. Set Up Development Environment: Install Xcode, which is the integrated development environment (IDE) for macOS. Xcode provides all the necessary tools and resources for developing Mac apps.\n\n3. Design the User Interface: Use Interface Builder in Xcode to design the user interface of your app. You can drag and drop UI elements, customize their properties, and create a visually appealing interface.\n\n4. Write Code: Use the programming language of your choice (Swift or Objective-C) to write the code for your app's functionality. This includes implementing features, handling

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.document_loaders import PyPDFLoader
llm = OpenAI(temperature=0)

ValidationError: 1 validation error for OpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
#loading the excel files
loader = UnstructuredExcelLoader(
    "excel/database.xlsx"
)

docs = loader.load()

loader = PyPDFLoader(
    "prof.pdf"
)

docs = loader.load()

In [ ]:
text_splitter_character = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100
)

 
#docs = TextLoader('file.txt').load()
#embedding
texts = text_splitter_character.split_documents(docs)

In [ ]:

embeddings = OpenAIEmbeddings()
state_of_union_store = Chroma.from_documents(
    texts, embeddings, collection_name="state-of-union"
)

In [ ]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)

vectorstore_info = VectorStoreInfo(
    name="state_of_union_address",
    description="the datails about plaksha professors and founders",
    vectorstore=state_of_union_store,
)

In [ ]:

toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)
agent_executor = create_vectorstore_agent(llm=llm, toolkit=toolkit, verbose=True)

In [ ]:
agent_executor.run(
    "Who can I meet to discuss about aeronautics"
)



> Entering new  chain...


OutputParserException: Could not parse LLM output: `I don't know.`

In [ ]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain

In [ ]:
#loading the excel files
loader = UnstructuredExcelLoader(
    "excel/database.xlsx"
)

docs = loader.load()

In [ ]:
text_splitter_character = RecursiveCharacterTextSplitter(
    chunk_size=10000, chunk_overlap=1000
)

 
#docs = TextLoader('file.txt').load()
#embedding
texts = text_splitter_character.split_documents(docs)

In [ ]:

embeddings = OpenAIEmbeddings()
plaksha_db = Chroma.from_documents(
    texts, embeddings, collection_name="plaksha"
)

In [ ]:
plaksha = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=plaksha_db.as_retriever()
)

In [ ]:
tools = [
       Tool(
        name="plaksha professor and founders",
        func=plaksha.run,
        description="useful when you need to recommend a professor or plaksha founder",
    ),
]

In [ ]:
prefix = """You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and execute these projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well. It suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, provideresources, roadmaps, advice, and suggest specialized tools or equipment required for implementation.

Now, as ConnectToLearn, your role is to help students discover projects aligned with their interests. To achieve this, please ask the following questions:

0. Do you already have a specific project idea or research topic in mind? If yes, please provide a brief description.

Based on the answer to the above question, proceed as follows:

- If the answer is 'Yes,' just ask them to provide short detail about the project afterthat, do not ask follow-up question again, you must provide a detailed step-by-step guidance on how they can execute their project idea or research topic, acquire the necessary resources, and go further in their project implementation.


- If the answer is 'No,' or the user does not have any project in mind, ensure to ask all this following general questions and add numbers to them, also user not necessarily have to answer all the questions:

1. What are some completed courses in which the you have taken? Please provide the course names or subjects.
2. Have there been specific subjects or topics that sparked the your curiosity during your studies? If yes, please specify.
3. Could the you share any areas of interest or passions you would like to explore further through projects?
4. Is there any particular expertise or guidance the you are seeking? For example, specific fields, industry connections, or project requirements.
5. Are there any resources, tools, or equipment that the you would require for the project implementation?
6. Do you prefer projects that are more research-oriented or hands-on implementation-based?
7. Are there any preferences regarding the types of organizations or experts the you would like to collaborate with?
8. What specific goals or outcomes do you hope to achieve through these projects?
9. Are there any constraints or limitations that need to be considered, such as time availability, budget, or access to certain resources?
10. How would the you describe your preferred learning style and approach to project-based learning?
11. Lastly, do you want a project, research or just the roadmap?


Once you have gathered the student's responses, as ConnectToLearn, your objective is to you must provide a detailed series of projects that can help them become experts in their chosen area. You must recommend multiple projects that align with their interests and goals, building upon their existing knowledge and skills. Additionally, offer detailed step-by-step guidance on how they can execute their project idea or research topic, acquire the necessary resources, and go further in their project implementation.


An example of recommended roadmaps is for web-development:-
Complete Frontend Developer Roadmap [With Resources] Step 1: Learn HTML Step 2: Learn CSS Step 3: Learn JavaScript Step 4: Learn Git and GitHub Step 5: CSS Architecture and Preprocessors Step 6: Learn Framework [React Recommend] Step 7: Modern CSS Step 8: CSS Frameworks [ Tailwind recommend] Step 9: Learn Authentication Strategies Step 10: Progressive Web Apps [optional] Step 11: Server-Side Rendering [Next Js, Nuxt Js] (optional) [Free Resources At the End] - For Html HTML is the backbone of any website. It's responsible for the structure and content of the webpage. Here's what to focus on: - Basic HTML syntax - Forms and inputs - Understanding the DOM (Document Object Model) - Semantic HTML (how to structure your HTML to convey meaning and improve accessibility) - SEO basics - For CSS CSS is what makes websites look good. It's all about the design, layout, and variations in display for different devices and screen sizes. - Basic CSS syntax - CSS box model - Flexbox and CSS Grid for layout designs - Responsive design principles - Animation with CSS - CSS variables - Learn CSS Preprocessors (Sass or Less) Preprocessors add extra functionality to CSS like variables, mixins, and functions which make CSS more maintainable. - Setting up a preprocessor - Sass or Less syntax - Creating variables and mixins - Nesting CSS rules - For Javascript JavaScript is the scripting language used to make webpages interactive. It's essential to learn both the basics and the advanced parts of JavaScript. - Syntax and basic constructs - Asynchronous JavaScript (Promises, async/await) - ES6+ concepts (let/const, arrow functions, destructuring, template literals, modules etc.) - DOM Manipulation and events - JSON and data fetching (Ajax, fetch API) - Understanding the concept of closures, and prototypal inheritance - Learn Git and GitHub Git is a version control system that helps you to keep track of changes made to the project. GitHub is a cloud-based hosting service that lets you manage Git repositories. -Basic command line commands - Basic Git commands (add, commit, push, pull) - Branching and merging - Resolving merge conflicts - Using GitHub (creating repositories, pull requests, and issues) - Learn a JavaScript Framework/Library (React recommended) React is a popular JavaScript library for building interactive UI components. - Understanding the virtual DOM - React components (Functional and Class) - State and props - Component lifecycle methods (for class components) or hooks (for functional components) - Making API calls in React - Routing with React Router - State management with Context API, Redux, or MobX (knowing one is generally enough, but each has its own use cases) - Modern CSS (CSS-in-JS, CSS Modules) Modern front-end development often involves component-based styles. - CSS Modules - Styled Components - Emotion - JSS - CSS Frameworks (Tailwind CSS recommended) Tailwind CSS is a utility-first CSS framework packed with classes like flex, pt-4, text-center and rotate-90 that can be composed to build any design, directly in your markup. - Understanding utility-first CSS - Tailwind CSS syntax - Configuring Tailwind CSS - Responsiveness in Tailwind CSS - Learn Authentication Strategies Most apps will require user authentication in some form. - OAuth - JWT - Understanding sessions - Implementing authentication in React Resources Html and CSS (freecodecamp) - https://youtu.be/G3e-cpL7ofc CSS - https://youtu.be/OXGznpKZ_sA SCSS and LESS - https://youtu.be/_a5j7KoflTs Javascript https://youtu.be/SBmSRK3feww Javascript Projects https://youtube.com/playlist?list=PLjVLYmrlmjGcZJ0oMwKkgwJ8XkCDAb9aG The Best Way To Learn Is Building projects. You have access to the following tools:"""
suffix = """Begin!  Use lots of "Args"



Question: {input}
{agent_scratchpad}"""

In [ ]:
prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)

In [ ]:
print(prompt.template)

You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and execute these projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well. It suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, provideresources, roadmaps, advice, and suggest specialized tools

In [ ]:
agent_executor.run("Plaksha professor that has experience in quantum mechanics")



> Entering new  chain...
 I need to find a source that can answer this question
Action: state_of_union_address_with_sources
Action Input: Plaksha professor that has experience in quantum mechanics
Observation: {"answer": " Dr. Tapas Pandit is a Plaksha professor that has experience in quantum mechanics.\n", "sources": "prof.pdf"}
Thought: I now know the final answer
Final Answer: Dr. Tapas Pandit is a Plaksha professor that has experience in quantum mechanics.

> Finished chain.


'Dr. Tapas Pandit is a Plaksha professor that has experience in quantum mechanics.'

In [ ]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [ ]:

tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [ ]:
agent_executor.run("hello")



> Entering new  chain...
Thought: What can I do to help the user?
Action: plaksha professor and founders
Action Input: user's area of interest

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 6804 tokens (6548 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.